In [28]:
import ee
import os
import geemap
import geopandas as gpd
from shapely import wkt
import numpy as np
from datetime import datetime
from datetime import timedelta
import pandas as pd
import matplotlib.pyplot as plt
import json
from qgrid import show_grid
from collections import Counter

In [47]:
out_dir = os.path.join(os.path.expanduser("~"), "Desktop","projects","glacier_segmentation","glims_data")
file_name = "master_glims_df.csv"
df = pd.read_csv(os.path.join(out_dir,file_name), index_col = 0) #read in data

In [48]:
df['date'] = [datetime.strptime(dat.split("T")[0], '%Y-%m-%d') for dat in list(df.src_date)] #convert string dates to datetime
df = df.loc[df['date'] > datetime(2005,1,1)] #select glaciers with polygons drawn after 2005

In [50]:
df = df[~df['geog_area'].isin(['Svalbard, Jan Mayen, and Bouvet', "Various (GlobGlacier)", 'Randolph Glacier Inventory; Umbrella RC for merging the RGI into GLIMS','Antarctic Peninsula','Various (NSIDC)'])]

In [8]:
df = df[(df.db_area > 1) & (df.db_area < 50)]

In [51]:
ids = list(Counter(df.glac_id).keys()) #get list of unique ids

In [54]:
df = df.reset_index(drop = True)

In [55]:
#create df which is a df of unique glaciers sorted by area, descending
clean_copy = df.copy()
s = set()
for i in range(df.shape[0]):
    print(i)
    if df.iloc[i,0] in s:
        clean_copy = clean_copy.drop(i, axis = 0)
        continue
    else:
        s.add(df.iloc[i,0])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [63]:
df = clean_copy #get top 3000 largest glaciers
df_sub = clean_copy

In [59]:
# Trigger the authentication flow
ee.Authenticate()

# Initialize the library
ee.Initialize()

Enter verification code: 4/1AX4XfWhD2Lm_JYpKF8qF95fw3sEGU7hHhV30qXeoMn4fYmupqC_jPA9pRmI

Successfully saved authorization token.


In [67]:
print(df.shape)

(474, 9)


In [65]:
out_dir = os.path.join(os.path.expanduser('~'),"Desktop","projects","glacier_segmentation", "nasa_dem")

for idx, glims_id in enumerate(df_sub.glac_id):
    one_glacier = df_sub.loc[df_sub.glac_id == glims_id]

    bbox = one_glacier.bboxes
    bbox = eval(list(bbox)[0])
    region = ee.Geometry.Polygon(list(bbox))
    
    filename = "{}_NASADEM.tif".format(glims_id) 
    DEM = ee.Image("NASA/NASADEM_HGT/001")
    DEM = DEM.clip(region)
    DEM = DEM.select(['elevation'])
    
    geemap.ee_export_image(DEM, filename = os.path.join(out_dir, filename), scale = 30, region = region, file_per_band = False)
    #900 of 19900 didn't get exported.                        

Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G212501E61399N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G079787E42137N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G079990E41931N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G076020E35955N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G074991E35994N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G080334E42156N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\m

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G071768E38938N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G074166E36631N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G074310E36745N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G211481E60940N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G079956E42288N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G016528E77664N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segme

Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G211009E61198N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G072826E39103N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G222222E59694N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G223301E58723N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G212137E61253N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G077277E43048N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projec

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G289986E32570S_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G101855E29563N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G086949E27913N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G323453E54383S_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G212185E61288N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G078291E41798N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segme

Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G043005E43089N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G323774E54434S_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G289994E32681S_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G212113E61220N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G229978E56520N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G007930E46472N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projec

...

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G238908E48572N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G237201E50137N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G073199E37935N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G238232E46829N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G072867E39103N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G072732E38828N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segme

Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G079485E44891N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G006872E45809N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G073785E38955N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G072525E37998N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G072916E38878N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G080633E30324N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projec

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G011715E46972N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G235508E51231N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G007380E45524N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G246276E48753N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G012219E47064N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G077749E41787N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segme

Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G070900E39023N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G087674E50129N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G018603E67938N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G074574E36571N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G007887E45948N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G007265E45549N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projec

Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G236021E50479N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G008087E46260N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G073081E38118N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G009919E46350N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G010865E46154N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segmentation\data_product_nasa_dem\G072701E38850N_NASADEM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\mattw\Desktop\projects\glacier_segme